# Generative Insights with BigQuery SQL and Vertex AI

## Step 1: Configuration and setup

In [3]:
import subprocess
import sys
import pkg_resources
from google.cloud import bigquery

# List of packages to install
packages = [
    'google-auth',
    'google-auth-oauthlib',
    'google-auth-httplib2',
    'matplotlib',
    'google-api-python-client',
    'google-cloud-aiplatform',
    'langchain==0.0.340',
    'chromadb==0.4.13',
    'google-cloud-bigquery'
]

# Function to check if a package is installed
def is_package_installed(package_name):
    try:
        pkg_resources.require(package_name)
        return True
    except pkg_resources.DistributionNotFound:
        return False
    except pkg_resources.VersionConflict:
        # In case of version conflicts, treat it as not installed
        return False

# Function to install packages
def install_packages(package_list):
    installed_packages = []
    newly_installed_packages = []
    
    for package in package_list:
        package_name = package.split('==')[0]  # Handle versioned packages
        if is_package_installed(package_name):
            installed_packages.append(package)
        else:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
            newly_installed_packages.append(package)
    
    return installed_packages, newly_installed_packages

# Install the packages
installed_packages, newly_installed_packages = install_packages(packages)

# Output based on the installation result
if newly_installed_packages:
    print("The following packages were installed:")
    for package in newly_installed_packages:
        print(f" - {package}")
else:
    print("All packages are already installed.")

###############################################################################


import os
# Change to D drive
os.chdir('D:/gcp-proj-ai-ml')


from google.oauth2 import service_account

# Path to your service account key file
service_account_file = 'D:/gcp-proj-ai-ml/sa-jupyter-gcloud.json'

# Authenticate using the service account key file
credentials = service_account.Credentials.from_service_account_file(service_account_file)

from googleapiclient.discovery import build
from google.oauth2 import service_account
from prettytable import PrettyTable

# Path to your service account key file
SERVICE_ACCOUNT_FILE = 'D:/gcp-proj-ai-ml/sa-jupyter-gcloud.json'

# Define the scopes required
SCOPES = ['https://www.googleapis.com/auth/cloud-platform']

# Define the project ID
PROJECT_ID = 'project-ai-ml-434700'


C:\Users\anuj dubey\AppData\Local\Temp\ipykernel_24092\1650153365.py:3: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


All packages are already installed.


## Preparing Data

In [4]:
# use the bq mk command to create a dataset called "movie_insights"
!bq mk --location=us-central1 movie_insights

# Clone the source file to you Cloud Shell Machine
!git clone https://github.com/AbiramiSukumaran/movie_score_genai_insights

# change the current directory to the git downloaded directory
!cd movie_score_genai_insights


Dataset 'project-ai-ml-434700:movie_insights' successfully created.


fatal: destination path 'movie_score_genai_insights' already exists and is not an empty directory.


## Navigate to the new project directory that is created and load the csv to the BQ dataset created

In [6]:
# Navigate to the new project directory that is created

import os
# Change to D drive
os.chdir('D:/gcp-proj-ai-ml/movie_score_genai_insights')

# Use the bq load command to load your CSV file into a BigQuery table

!bq load --source_format=CSV --skip_leading_rows=1 \
movie_insights.movie_score \
./movies_data.csv \
Id:numeric,name:string,rating:string,genre:string,year:numeric,released:string,score:string,director:string,writer:string,star:string,country:string,budget:numeric,company:string,runtime:numeric,data_cat:string


# You can query a sample to check if the table movie_score and the data are created in the dataset

!bq query --use_legacy_sql=false \
SELECT name, rating, genre, runtime FROM movie_insights.movie_score limit 3;


Upload complete.

Waiting on bqjob_r1a02899163520087_00000191f1aeb2a1_1 ... (0s) Current status: RUNNING
                                                                                      
Waiting on bqjob_r1a02899163520087_00000191f1aeb2a1_1 ... (0s) Current status: DONE   


+------------------+-----------+--------+---------+
|       name       |  rating   | genre  | runtime |
+------------------+-----------+--------+---------+
| Love by Drowning | R         | Drama  |     121 |
| It's Just Us     | Not Rated | Drama  |     120 |
| The Robinsons    | Not Rated | Action |      90 |
+------------------+-----------+--------+---------+


In [1]:
# # You can query a sample to check if the table movie_score and the data are created in the dataset

# !bq query --use_legacy_sql=false \
# SELECT name, rating, genre, runtime FROM movie_insights.movie_score limit 5;

+------------------+-----------+--------+---------+
|       name       |  rating   | genre  | runtime |
+------------------+-----------+--------+---------+
| Love by Drowning | R         | Drama  |     121 |
| It's Just Us     | Not Rated | Drama  |     120 |
| The Robinsons    | Not Rated | Action |      90 |
| Glitter          | PG-13     | Drama  |     104 |
| Leonard Part 6   | PG        | Action |      85 |
+------------------+-----------+--------+---------+


## Create a classification model to predict the success score of the movie based on GENRE and RUNTIME attributes.

In [7]:
client = bigquery.Client()

QUERY_TRAIN = ("""
        CREATE OR REPLACE MODEL
  `movie_insights.model_rating_by_runtime_genre_01`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    data_split_method='NO_SPLIT',
    model_registry='vertex_ai',   
    vertex_ai_model_version_aliases=['logistic_reg', 'experimental'],
    input_label_cols=['score']
  ) AS
SELECT name, genre,runtime, score
FROM
  movie_insights.movie_score
WHERE
  data_cat = 'TRAIN';
          """)

query_job_train = client.query(QUERY_TRAIN)
# df_train = query_job_train.to_dataframe()
# df_train.head()

## Predicting movie score using the model

In [10]:
# Predicting movie score using the model

client = bigquery.Client()

QUERY_PREDICT = ("""
        SELECT
          *
        FROM
          ML.PREDICT (MODEL movie_insights.model_rating_by_runtime_genre,
            (
            SELECT
              *
            FROM
              movie_insights.movie_score
            WHERE
              data_cat= 'TEST'
             )
          );
          """)

query_job_predict = client.query(QUERY_PREDICT)
df_prediction = query_job_predict.to_dataframe()
df_prediction.head()

C:\Users\anuj dubey\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,predicted_score,predicted_score_probs,Id,name,rating,genre,year,released,score,director,writer,star,country,budget,company,runtime,data_cat
0,7,"[{'label': '7', 'prob': 0.337056592816488}, {'...",7641.000000000,By Light of Desert Night,Not Rated,Drama,2019.000000000,3/3/2020,5,David Stuart Snell,David Stuart Snell,Alexandra Bokova,United States,57531050.000000000,Canyon Media,94.000000000,TEST
1,7,"[{'label': '7', 'prob': 0.3382462144242809}, {...",7656.000000000,Tulsa,PG-13,Comedy,2020.000000000,6/3/2020,5,Scott Pryor,Scott Pryor,Scott Pryor,United States,57531050.000000000,Pryor Entertainment,120.000000000,TEST
2,7,"[{'label': '7', 'prob': 0.3602429747290255}, {...",7655.000000000,Legend of Deification,TV-PG,Animation,2020.000000000,10/1/2020,7,Teng Cheng,Jiang Ziya,Guangtao Jiang,China,57531050.000000000,Beijing Enlight Pictures,110.000000000,TEST
3,7,"[{'label': '7', 'prob': 0.39737381968856844}, ...",7642.000000000,A Brother's Love,Not Rated,Drama,2019.000000000,6/7/2019,7,Monia Chokri,Monia Chokri,Anne-Élisabeth Bossé,Canada,57531050.000000000,Metafilms,117.000000000,TEST
4,6,"[{'label': '6', 'prob': 0.37572885944327344}, ...",7649.000000000,Sonic the Hedgehog,PG,Action,2020.000000000,2/14/2020,7,Jeff Fowler,Pat Casey,Ben Schwartz,United States,85000000.000000000,Paramount Pictures,99.000000000,TEST


## Create an External Connection

In [11]:
# Create an External Connection

!bq mk --connection --location=us-central1 --project_id=project-ai-ml-434700 \
    --connection_type=CLOUD_RESOURCE bq_llm_connection

Connection 235498471165.us-central1.bq_llm_connection successfully created


## Create a remote ML model

<span style="color:red">Important</span> : Before running the step, go to the detail page of the "external connection" created and note down the SA. Then give vertex AI User" role to that Service account(SA)

In [12]:
# Create a remote ML model

QUERY_REMOTE_MODEL = ("""
        CREATE OR REPLACE MODEL
  movie_insights.llm_model REMOTE
WITH CONNECTION `us-central1.bq_llm_connection` OPTIONS (remote_service_type = 'CLOUD_AI_LARGE_LANGUAGE_MODEL_V1');
          """)

query_create_remote_model = client.query(QUERY_REMOTE_MODEL)

## Generate text using the ML model

In [18]:
QUERY_CREATE_TEXT = ("""
        SELECT
  ml_generate_text_result['predictions'][0]['content'] AS generated_text,
  ml_generate_text_result['predictions'][0]['safetyAttributes']
    AS safety_attributes,
  * EXCEPT (ml_generate_text_result)
FROM
  ML.GENERATE_TEXT(
    MODEL `movie_insights.llm_model`,
    (
 SELECT
      CONCAT('FROM THE FOLLOWING TEXT ABOUT MOVIES, WHAT DO YOU THINK ARE THE FACTORS INFLUENCING A MOVIE SCORE TO BE GREATER THAN 5?: ', movie_data) AS prompt
    FROM (
      SELECT
        REPLACE(STRING_AGG( CONCAT('A movie named ',name, ' from the country ', country, ' with a censor rating of ',rating, ' and a budget of ', budget, ' produced by ', company, ' with a runtime of about ', runtime, ' and in the genre ', genre, ' starring ', star, ' has had a success score of ', score, '') ), ',','. ') AS movie_data
      FROM (
        SELECT
          *
        FROM
          `movie_insights.movie_score`
        WHERE
          CAST(SCORE AS INT64) > 5
        LIMIT
          50) ) AS MOVIES
    ),
    STRUCT(
      0.2 AS temperature,
      100 AS max_output_tokens));
          """)

# Function to wrap text to fit within column width
def wrap_text(text, width):
    # Word-wrap text manually by splitting into lines
    wrapped = "\n".join([text[i:i+width] for i in range(0, len(text), width)])
    return wrapped

query_create_text = client.query(QUERY_CREATE_TEXT)
df_gen_text = query_create_text.to_dataframe()

column_width = 30  # Around 2 inches (based on a character count of around 10-12 characters per inch)

# Initialize PrettyTable
table = PrettyTable()

# Add column names from the DataFrame
table.field_names = df_gen_text.columns

# Add rows to the PrettyTable with word wrapping applied to each cell
for row in df_gen_text.itertuples(index=False, name=None):
    wrapped_row = [wrap_text(str(cell), column_width) for cell in row]
    table.add_row(wrapped_row)

# Optional: Set alignment for all columns to be left-aligned
table.align = "l"

# Display the table
print(table)


+--------------------------------+--------------------------------+-------------------------+--------------------------------+
| generated_text                 | safety_attributes              | ml_generate_text_status | prompt                         |
+--------------------------------+--------------------------------+-------------------------+--------------------------------+
|  Based on the provided informa | {'blocked': False, 'categories |                         | FROM THE FOLLOWING TEXT ABOUT  |
| tion, it is difficult to deter | ': array(['Derogatory', 'Finan |                         | MOVIES, WHAT DO YOU THINK ARE  |
| mine the exact factors influen | ce', 'Insult', 'Profanity', 'S |                         | THE FACTORS INFLUENCING A MOVI |
| cing a movie score to be great | exual', 'Toxic'],              |                         | E SCORE TO BE GREATER THAN 5?: |
| er than 5. However, some poten |       dtype=                   |                         |  A movie named Am

## Flatten the query result

In [19]:
QUERY_FLATTEN_TEXT = ("""
        SELECT
  *
FROM
  ML.GENERATE_TEXT( MODEL movie_insights.llm_model,
    (
    SELECT
      CONCAT('FROM THE FOLLOWING TEXT ABOUT MOVIES, WHAT DO YOU THINK ARE THE FACTORS INFLUENCING A MOVIE SCORE TO BE GREATER THAN 5?: ', movie_data) AS prompt
    FROM (
      SELECT
        REPLACE(STRING_AGG( CONCAT('A movie named ',name, ' from the country ', country, ' with a censor rating of ',rating, ' and a budget of ', budget, ' produced by ', company, ' with a runtime of about ', runtime, ' and in the genre ', genre, ' starring ', star, ' has had a success score of ', score, '') ), ',','. ') AS movie_data
      FROM (
        SELECT
          *
        FROM
          `movie_insights.movie_score`
        WHERE
          CAST(SCORE AS INT64) > 5
        LIMIT
          50) ) AS MOVIES),
    STRUCT( 0.2 AS temperature,
      100 AS max_output_tokens,
      TRUE AS flatten_json_output));
          """)




# Function to wrap text to fit within column width
# def wrap_text(text, width):
#     # Word-wrap text manually by splitting into lines
#     wrapped = "\n".join([text[i:i+width] for i in range(0, len(text), width)])
#     return wrapped

query_flatten_text = client.query(QUERY_FLATTEN_TEXT)
df_flat_text = query_flatten_text.to_dataframe()

column_width = 30  # Around 2 inches (based on a character count of around 10-12 characters per inch)

# Initialize PrettyTable
table = PrettyTable()

# Add column names from the DataFrame
table.field_names = df_flat_text.columns

# Add rows to the PrettyTable with word wrapping applied to each cell
for row in df_flat_text.itertuples(index=False, name=None):
    wrapped_row = [wrap_text(str(cell), column_width) for cell in row]
    table.add_row(wrapped_row)

# Optional: Set alignment for all columns to be left-aligned
table.align = "l"

# Display the table
print(table)


C:\Users\anuj dubey\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:207: UserWarning: Unable to determine type for field 'ml_generate_text_rai_result'.
  warnings.warn("Unable to determine type for field '{}'.".format(bq_field.name))


+--------------------------------+--------------------------------+-------------------------+--------------------------------+
| ml_generate_text_llm_result    | ml_generate_text_rai_result    | ml_generate_text_status | prompt                         |
+--------------------------------+--------------------------------+-------------------------+--------------------------------+
|  Based on the provided informa | {'blocked': False, 'categories |                         | FROM THE FOLLOWING TEXT ABOUT  |
| tion, it is difficult to deter | ': array(['Derogatory', 'Finan |                         | MOVIES, WHAT DO YOU THINK ARE  |
| mine the exact factors influen | ce', 'Insult', 'Profanity', 'S |                         | THE FACTORS INFLUENCING A MOVI |
| cing a movie score to be great | exual', 'Toxic'],              |                         | E SCORE TO BE GREATER THAN 5?: |
| er than 5. However, some poten |       dtype=                   |                         |  A movie named Am